In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%matplotlib inline
%pylab inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))
import warnings
warnings.filterwarnings(action='ignore')
# Any results you write to the current directory are saved as output.


    DBNOs - Number of enemy players knocked.
    assists - Number of enemy players this player damaged that were killed by teammates.
    boosts - Number of boost items used.
    damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.
    headshotKills - Number of enemy players killed with headshots.
    heals - Number of healing items used.
    killPlace - Ranking in match of number of enemy players killed.
    killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.)
    killStreaks - Max number of enemy players killed in a short amount of time.
    kills - Number of enemy players killed.
    longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.
    matchDuration - Duration of match in seconds
    matchId - Integer ID to identify match. There are no matches that are in both the training and testing set.
    matchType - String identifying the game mode that the data comes from. The standard modes are “solo”, “duo”, “squad”, “solo-fpp”, “duo-fpp”, and “squad-fpp”; other modes are from events or custom matches. 
    revives - Number of times this player revived teammates.
    rideDistance - Total distance traveled in vehicles measured in meters.
    roadKills - Number of kills while in a vehicle.
    swimDistance - Total distance traveled by swimming measured in meters.
    teamKills - Number of times this player killed a teammate.
    vehicleDestroys - Number of vehicles destroyed.
    walkDistance - Total distance traveled on foot measured in meters.
    weaponsAcquired - Number of weapons picked up.
    winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.)
    groupId - Integer ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
    numGroups - Number of groups we have data for in the match.
    maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
    winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.


In [ ]:
plt.figure(figsize = (20, 20))

In [ ]:
df = pd.read_csv('../input/train_V2.csv')

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
def plot_counts(p):
    plt.figure(figsize = (20, 15))
    sns.countplot(df[p]).set_title(p)
    plt.show()

In [ ]:
def plot_dists(p, b = 50, kde_flag = True, rug_flag = False):
    plt.figure(figsize = (20, 15))
    sns.distplot(df[p], kde= kde_flag, rug = rug_flag, bins = b).set_title(p)
    plt.show()

In [ ]:
def plot_scatters(x, y, title):
    plt.figure(figsize = (20, 15))
    sns.scatterplot(df[x], df[y]).set_title(title)
    plt.show()

**DBNOs**

In [ ]:
plot_counts("DBNOs")

**Boosts**

In [ ]:
plot_counts("boosts")

**Headshotskill**

In [ ]:
plot_counts("headshotKills")

**Heals**

In [ ]:
plot_counts("heals")

**Killstreaks**

In [ ]:
plot_counts("killStreaks")

**Kills**

In [ ]:
plot_counts("kills")

**Revives**

In [ ]:
plot_counts("revives")

**Roadkills**

In [ ]:
plot_counts("roadKills")

**Teamkills**

In [ ]:
plot_counts("teamKills")

**Vehicledestroys**

In [ ]:
plot_counts("vehicleDestroys")

**WeaponsAcquired**

In [ ]:
plot_counts("weaponsAcquired")

**WalkDistance**

In [ ]:
plot_dists('walkDistance')

**Match type**

In [ ]:
plot_counts("matchType")

**Headshots Rate**

In [ ]:
df["headshot_rate"] = df["headshotKills"] / df["kills"]
df["headshot_rate"].fillna(0, inplace = True)
plot_dists("headshot_rate")

**Roadkills Rate**

In [ ]:
df["roadkills_rate"] = df["roadKills"] / df["kills"]
df["roadkills_rate"].fillna(0, inplace = True)
plot_dists("roadkills_rate")

**Kill place **

In [ ]:
plot_dists("killPlace")

**Swin Distance**

In [ ]:
plot_dists("swimDistance")

**Longestkill**

In [ ]:
plot_dists("longestKill")

**Distance travelled and road kills**

In [ ]:
plot_scatters("rideDistance", "roadKills", "RoadKills by RideDistance")

**KillPlace**

In [ ]:
plot_dists("killPlace")

**Kills and Killplace**

In [ ]:
plot_scatters("killPlace", "kills", "Kills by Killplace")

**WinPlacePerc and Kills**

In [ ]:
plot_scatters("winPlacePerc", "kills", "No of kills by winper")

**WinPlacePerc and Match duration**

In [ ]:
df["matchDurationMinute"] = df["matchDuration"].apply(lambda x: x/60)
plot_scatters("winPlacePerc", "matchDurationMinute", "Wins by match duration")

In [ ]:
plot_scatters("winPlacePerc","killPlace","")

No of kills increase the chances of wins

**Heals/Boost and WinPer**

In [ ]:
plt.figure(figsize = (20, 15))
sns.pointplot(df["heals"], df["winPlacePerc"], linestyles="-")
sns.pointplot(df["boosts"], df["winPlacePerc"], color = "green", linestyles="--")
plt.xlabel("heals/boost")
plt.legend(["heals","boosts"]) 
plt.show()

No. of Boosts increases the chances of win than no. of heals

** Number of enemy players knocked by  Number of enemy players this player damaged that were killed by teammates.**

In [ ]:
plt.figure(figsize = (20, 15))
sns.pointplot(df["DBNOs"], df["assists"])
plt.grid()
plt.show()

**Heals/Boost and walk distance**

In [ ]:
plt.figure(figsize = (20, 15))
sns.pointplot(df["heals"], df["walkDistance"], linestyles="-")
sns.pointplot(df["boosts"], df["walkDistance"], color = "green", linestyles="--")
plt.xlabel("heals/boost")
plt.legend(["heals","boosts"]) 
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize = (20, 15))
sns.heatmap(df.corr(), annot = True, fmt='.1f')
plt.show()

In [ ]:
del df

In [ ]:
train_df = pd.read_csv("../input/train_V2.csv")
test_df = pd.read_csv("../input/test_V2.csv")

In [ ]:
matchTyp = ['squad-fpp', 'duo', 'solo-fpp', 'squad', 'duo-fpp', 'solo',
       'normal-squad-fpp', 'crashfpp', 'flaretpp', 'normal-solo-fpp',
       'flarefpp', 'normal-duo-fpp', 'normal-duo', 'normal-squad',
       'crashtpp', 'normal-solo']
mapping = {}
for i, j in enumerate(matchTyp):
    mapping[i] = j
train_df["matchTypeMap"] = train_df["matchType"].apply(lambda x: ''.join(str(i) for i, j in mapping.items() if x == j)).map(int64)
test_df["matchTypeMap"] = test_df["matchType"].apply(lambda x: ''.join(str(i) for i, j in mapping.items() if x == j)).map(int64)

In [ ]:
train_df.drop(["matchType"], axis =1, inplace=True)
test_df.drop(["matchType"], axis =1, inplace=True)

In [ ]:
train_df.dropna(inplace = True)
train_df.isnull().any().any()

In [ ]:
X = train_df.drop(["Id", "groupId", "matchId", "winPlacePerc"], axis = 1)
y = train_df["winPlacePerc"]
test = test_df.drop(["Id", "groupId", "matchId"], axis = 1)

In [ ]:
del train_df

In [ ]:
model = xgboost.XGBRegressor(max_depth=17, gamma=0.3, learning_rate= 0.1)
model.fit(X,y)
del X
del y

Plotting feature importance

In [ ]:
xgboost.plot_importance(model)

In [ ]:
pred = model.predict(test)

In [ ]:
test_id = test_df["Id"]
submit_xg = pd.DataFrame({'Id': test_id, "winPlacePerc": pred} , columns=['Id', 'winPlacePerc'])
print(submit_xg.head())
submit_xg.to_csv("submission.csv", index = False)

**To be continued [Progress]**